Trước khi run pip, cài đặt các thư viện bằng câu lệnh sau
```
pip install dash
```

In [1]:
# Cài đặt các thư viện cần thiết
import dash
from dash import dcc
from dash import  html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from keras.models import load_model
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

import numpy as np

# Khởi tạo ứng dụng Dash
app = dash.Dash()
server = app.server

# Đọc dữ liệu, sắp xếp theo ngày, lấy cột ngày và giá đóng cửa
df_nse = pd.read_csv("NSE-TATA.csv")

df_nse["Date"]=pd.to_datetime(df_nse.Date,format="%Y-%m-%d")
df_nse.index=df_nse['Date'].values

data=df_nse.sort_index(ascending=True,axis=0)
new_data=pd.DataFrame(index=range(0,len(df_nse)),columns=['Date','Close'])

for i in range(0,len(data)):
    new_data.loc[i, "Date"]=data['Date'].iloc[i]
    new_data.loc[i, "Close"]=data["Close"].iloc[i]

new_data.index=new_data['Date'].values
new_data.drop("Date",axis=1,inplace=True)
new_data.head()

# Chuẩn hóa dữ liệu
dataset=new_data.values

train=dataset[0:987,:]
valid=dataset[987:,:]

scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)

x_train,y_train=[],[]

for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])

x_train,y_train=np.array(x_train),np.array(y_train)

x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))



In [2]:
# Lấy mẫu từ dữ liệu để đưa ra dự đoán giá cổ phiếu bằng mô hình LSTM, RNN, XGB
lstm_model=load_model("saved_lstm_model.keras")
rnn_model=load_model("saved_rnn_model.keras")
xgb_model=xgb.Booster()
xgb_model.load_model("saved_xgboost_model.json")

inputs=new_data[len(new_data)-len(valid)-60:].values
inputs=inputs.reshape(-1,1)
inputs=scaler.transform(inputs)

X_test=[]
for i in range(60,inputs.shape[0]):
    X_test.append(inputs[i-60:i,0])
X_test=np.array(X_test)

X_test_3D=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

closing_price_lstm=lstm_model.predict(X_test_3D)
closing_price_lstm=scaler.inverse_transform(closing_price_lstm)

closing_price_rnn=rnn_model.predict(X_test_3D)
closing_price_rnn=scaler.inverse_transform(closing_price_rnn)

X_test_DMatrix=xgb.DMatrix(X_test)
closing_price_xgb=xgb_model.predict(X_test_DMatrix).reshape(-1, 1)
closing_price_xgb=scaler.inverse_transform(closing_price_xgb)

train=new_data[:987]
valid=new_data[987:]

valid.loc[:, 'Predictions_LSTM'] = closing_price_lstm
valid.loc[:, 'Predictions_RNN'] = closing_price_rnn
valid.loc[:, 'Predictions_XGB'] = closing_price_xgb

print(valid)

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
             Close  Predictions_LSTM  Predictions_RNN  Predictions_XGB
2017-10-09   208.3        200.764099       210.592285       199.185440
2017-10-10  208.45        201.603867       210.175156       201.694412
2017-10-11   209.4        202.387939       213.218170       199.840088
2017-10-12   212.0        203.160492       212.117661       199.329651
2017-10-13  210.25        204.110718       213.955109       199.519806
...            ...               ...              ...              ...
2018-10-01   230.9        230.380280       236.682098       232.618698
2018-10-03   227.6        230.146790       234.532166       233.537170
2018-10-04   218.2        229.292679       231.967026       229.404465
2018-10-05   209.2        227.178467       225.080063       218.499207
2018-10-08  215.15        223.620636       214.697418       214.501419

[248 rows x 4 columns]


C:\Users\hieu\AppData\Local\Temp\ipykernel_3184\1429423023.py:31: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hieu\AppData\Local\Temp\ipykernel_3184\1429423023.py:32: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\hieu\AppData\Local\Temp\ipykernel_3184\1429423023.py:33: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [3]:
df= pd.read_csv("stock_data.csv")

app.layout = html.Div([

    html.H1("Stock Price Analysis Dashboard", style={"textAlign": "center"}),

    dcc.Tabs(id="tabs", children=[

        dcc.Tab(label='NSE-TATAGLOBAL Actual closing price',children=[
			html.Div([
				html.H2("Actual closing price",style={"textAlign": "center"}),
				dcc.Graph(
					id="Actual Data",
					figure={
						"data":[
							go.Scatter(
								x=valid.index,
								y=valid["Close"],
								mode='markers'
							)

						],
						"layout":go.Layout(
							title='scatter plot',
							xaxis={'title':'Date'},
							yaxis={'title':'Closing Rate'}
						)
					}

				),
			])


        ]),
        dcc.Tab(label='NSE-TATAGLOBAL prediction closing price', children=[
            html.Div([
                html.H1("Prediction closing price",
                        style={'textAlign': 'center'}),

                dcc.Dropdown(id='my-dropdown',
                             options=[{'label': 'LSTM', 'value': 'LSTM'},
                                      {'label': 'RNN','value': 'RNN'},
                                      {'label': 'XGBoost', 'value': 'XGB'}],
                             multi=True,value=['LSTM'],
                             style={"display": "block", "margin-left": "auto",
                                    "margin-right": "auto", "width": "60%"}),
                dcc.Graph(id='prediction'),

            ]),
        ])


    ])
])

@app.callback(Output('prediction', 'figure'),
              [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown):
    dropdown = {"LSTM": "LSTM","RNN": "RNN", "XGB": "XGBoost"}
    trace1 = []
    trace2 = []
    
    for model in selected_dropdown:
        trace1.append(
          go.Scatter(x=valid.index,
					 y=valid["Predictions" + "_" + model],
                     mode='lines', opacity=0.7,
                     name=f'Predictions_{model}',textposition='bottom center'))
        

    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout(colorway=["#5E0DAC", '#FF4F00', '#375CB1',
                                            '#FF7400', '#FFF400', '#FF0056'],
            height=600,
            title=f"High and Low Prices for {', '.join(str(dropdown[i]) for i in selected_dropdown)} Over Time",
            xaxis={"title":"Date",
                   'rangeselector': {'buttons': list([{'count': 1, 'label': '1M',
                                                       'step': 'month',
                                                       'stepmode': 'backward'},
                                                      {'count': 6, 'label': '6M',
                                                       'step': 'month',
                                                       'stepmode': 'backward'},
                                                      {'step': 'all'}])},
                   'rangeslider': {'visible': True}, 'type': 'date'},
             yaxis={"title":"Price (USD)"})}
    return figure


if __name__=='__main__':
	app.run_server(debug=True)